In [5]:
#import IPython

#IPython.Application.instance().kernel.do_shutdown(True) #automatically restarts kernel
#import os
#os._exit(00)

#from IPython.display import display_html
#def restartkernel() :
#    display_html("<script>Jupyter.notebook.kernel.restart()</script>",raw=True)
    
#restartkernel()

#import IPython


#IPython.Application.instance().kernel.restart(True) #automatically restarts kernel

%reset -f

In [6]:
import redis
import json
from datetime import date
from datetime import timedelta

r = redis.Redis(host='localhost', port=6379, db=0)

dataCompany = json.loads(r.get('dataCompany')) 
cost_kWh_generation = float(dataCompany['priceSolarPanelGenerate'])

solarPanelGeneration = json.loads(r.get('solarPanelGeneration')) 
acum_kWh_generation = float(solarPanelGeneration['kWh_acumGeneration'])

prevDay = (date.today() - timedelta(days = 1)).strftime('%d/%m/%y')

#Generation history
data_setGenerationHistory = {"date":prevDay,"kWh":round(acum_kWh_generation,6),"benefit":round((acum_kWh_generation*cost_kWh_generation),3)}

json_dumpGenerationHistory = json.dumps(data_setGenerationHistory)
r.lpush('generationHistoryList',json_dumpGenerationHistory) 

#Reset generationTodayList
r.delete('generationTodayList')


0

In [7]:
import redis
import json
from datetime import date
from datetime import timedelta

r = redis.Redis(host='localhost', port=6379, db=0)

dataCompany = json.loads(r.get('dataCompany')) 
cost_kWh_consumption = float(dataCompany['pricekWh'])

energyConsumption = json.loads(r.get('energyConsumption')) 
acum_kWh_consumption = float(energyConsumption['kWh_acum'])

given_date = (date.today() - timedelta(days = 1))
#given_date = date(2019, 6, 23)
first_day_of_month = given_date.replace(day=1).strftime('%d/%m/%y')
 
#print("\nFirst day of month: ", first_day_of_month, "\n")

#Consumption history

#Buscamos si en la tabla hash esta la key del primer dia este mes.

returnRedis = r.hget('consumptionHistoryHashList', first_day_of_month)
#print(returnRedis)

if returnRedis!=None:
    #Se ha encontrado la key, por lo tanto actualizamos los kWh acum del mes.
    tempJson = json.loads(returnRedis)
    tempJson['kWh']+=round(acum_kWh_consumption,6)
    tempJson['cost']=round((tempJson['kWh']*cost_kWh_consumption),3)
    
    json_dumpConsumptionHistoryUPD = json.dumps(tempJson)
    r.hset('consumptionHistoryHashList',first_day_of_month,json_dumpConsumptionHistoryUPD)
    
else:
    #No se ha encontrado la key, por lo tanto creamos una nueva.
    print("No hay nada!")
    
    # Guaranteed to get the next month. Force any_date to 28th and then add 4 days.
    next_month = (date.today() - timedelta(days = 1)).replace(day=28) + timedelta(days=4)
    #next_month = date(2019, 6, 23).replace(day=28) + timedelta(days=4)
    # Subtract all days that are over since the start of the month.
    last_day_of_month = (next_month - timedelta(days=next_month.day)).strftime('%d/%m/%y')
    
    data_setConsumptionHistory = {"endPeriod":last_day_of_month,"kWh":round(acum_kWh_consumption,6),"cost":round((acum_kWh_consumption*cost_kWh_consumption),3)}
    json_dumpConsumptionHistory = json.dumps(data_setConsumptionHistory)
    r.hset('consumptionHistoryHashList',first_day_of_month,json_dumpConsumptionHistory)


In [8]:
pathDataCSV = 'data/REDD/low_freq2'
pathDataH5 = 'data/data.h5'

In [9]:
import redis
import json
r = redis.Redis(host='localhost', port=6379, db=0)

appliances = json.loads(r.get('dataAppliances')) 

selectedAppliances = []

if appliances['Refrigerator']==True:
    selectedAppliances.append('fridge')
if appliances['Dryer']==True:
    selectedAppliances.append('washer dryer')
if appliances['Microwave']==True:
    selectedAppliances.append('microwave')
if appliances['Dishwasher']==True:
    selectedAppliances.append('dish washer')
if appliances['Oven']==True:
    selectedAppliances.append('electric oven')
if appliances['Electric Hob']==True:
    selectedAppliances.append('electric stove')
if appliances['Electric heater']==True:
    selectedAppliances.append('electric space heater')
    
print(selectedAppliances)

['fridge', 'microwave', 'electric oven']


In [10]:
import csv
rtList = r.lrange("rtList2", 0, -1);

rtList.reverse()


with open(pathDataCSV+'/house_1/Generadochannel_1Mal.dat', 'w', newline='') as file:
    writer = csv.writer(file, delimiter=' ')
    for i in rtList:
        data = json.loads(i)
        writer.writerow([data['epochTime'], round(data['kW']*1000,2)])

In [ ]:
#from nilmtk.dataset_converters import convert_redd
from nilmtk.dataset_converters import convert_reddAlex
convert_reddAlex(pathDataCSV, pathDataH5)
#convert_redd('data/REDD/dades', 'data/dades.h5')

Loading house 1... 1 3 4 5 6 7 8 9 

In [ ]:
from __future__ import print_function, division
import time

from matplotlib import rcParams
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from six import iteritems

from nilmtk import DataSet, TimeFrame, MeterGroup, HDFDataStore
from nilmtk.legacy.disaggregate import CombinatorialOptimisation, FHMM
import nilmtk.utils

%matplotlib inline

In [ ]:
rcParams['figure.figsize'] = (13, 6)

In [ ]:
train = DataSet(pathDataH5)
test = DataSet(pathDataH5)

#print(test.buildings[1].elec.mains().load(physical_quantity = 'power', ac_type = 'apparent', sample_period=1))

In [ ]:
building = 1

In [ ]:
from datetime import date
from datetime import timedelta

day = (date.today() - timedelta(days = 1)).strftime('%Y-%m-%d')

print("Date to predict appliances consuption"+day)

#train => Model
#test => Data meter

#train.set_window(end=day)
#test.set_window(start=day)

#train.set_window(end="2021-10-20")
#test.set_window(start="2021-10-20")

train.set_window(end="2011-05-01")
test.set_window(start="2011-05-01")

train_elec = train.buildings[1].elec
test_elec = test.buildings[1].elec

In [ ]:
train_elec.plot()

In [ ]:
test_elec.mains().plot()

In [ ]:
def predict(clf, test_elec, sample_period, timezone):
    pred = {}
    gt= {}

    # "ac_type" varies according to the dataset used. 
    # Make sure to use the correct ac_type before using the default parameters in this code.    
    for i, chunk in enumerate(test_elec.mains().load(physical_quantity = 'power', ac_type = 'apparent', sample_period=sample_period)):
        chunk_drop_na = chunk.dropna()
        pred[i] = clf.disaggregate_chunk(chunk_drop_na)
        gt[i]={}

        for meter in test_elec.submeters().meters:
            # Only use the meters that we trained on (this saves time!)    
            gt[i][meter] = next(meter.load(physical_quantity = 'power', ac_type = 'active', sample_period=sample_period))
        gt[i] = pd.DataFrame({k:v.squeeze() for k,v in iteritems(gt[i]) if len(v)}, index=next(iter(gt[i].values())).index).dropna()
        
    # If everything can fit in memory
    gt_overall = pd.concat(gt)
    gt_overall.index = gt_overall.index.droplevel()
    pred_overall = pd.concat(pred)
    pred_overall.index = pred_overall.index.droplevel()

    # Having the same order of columns
    gt_overall = gt_overall[pred_overall.columns]
    
    #Intersection of index
    gt_index_utc = gt_overall.index.tz_convert("UTC")
    pred_index_utc = pred_overall.index.tz_convert("UTC")
    common_index_utc = gt_index_utc.intersection(pred_index_utc)
    
    common_index_local = common_index_utc.tz_convert(timezone)
    gt_overall = gt_overall.loc[common_index_local]
    pred_overall = pred_overall.loc[common_index_local]
    appliance_labels = [m for m in gt_overall.columns.values]
    gt_overall.columns = appliance_labels
    pred_overall.columns = appliance_labels
    return gt_overall, pred_overall

In [ ]:
#top_5_train_elec = train_elec.submeters().select_top_k(k=2)
top_5_train_elec = train_elec.submeters().select_using_appliances(type=selectedAppliances)

classifiers = {'CO':CombinatorialOptimisation(), 'FHMM':FHMM()}
predictions = {}
sample_period = 120 #Cambiar el sample_period a 1, cuando este en produccion
for clf_name, clf in classifiers.items():
    print("*"*20)
    print(clf_name)
    print("*" *20)
    start = time.time()
    # Note that we have given the sample period to downsample the data to 1 minute. 
    # If instead of top_5 we wanted to train on all appliance, we would write 
    # fhmm.train(train_elec, sample_period=60)
    clf.train(top_5_train_elec, sample_period=sample_period)
    end = time.time()
    print("Runtime =", end-start, "seconds.")
    gt, predictions[clf_name] = predict(clf, test_elec, sample_period, train.metadata['timezone'])

In [ ]:
appliance_labels = [m.label() for m in gt.columns.values]

In [ ]:
gt.columns = appliance_labels
predictions['CO'].columns = appliance_labels
predictions['FHMM'].columns = appliance_labels
#Create DataFrame
df = pd.DataFrame({predictions['CO'].head(25).to_string()})

#Convert the Columns to string
df[df.columns]=df[df.columns].astype(str)

#Create the list of column separator width 
SepWidth = [5,6,3,8]

#Temp dict
tempdf = {}
#Convert all the column to series
for i, eCol in enumerate(df):
    tempdf[i] = pd.Series(df[eCol]).str.pad(width=SepWidth[i])

#Final DataFrame
Fdf = pd.concat(tempdf, axis=1)
#print Fdf
#Export to csv
Fdf.to_csv("./testCO.txt", sep='\t', index=False, header=False, encoding='utf-8')




In [ ]:
predictions['CO'].head(25)

In [ ]:
predictions['FHMM'].head(25)

In [ ]:
gt.head(25)

In [ ]:
print(predictions['FHMM'])

In [ ]:
import math

#day = (date.today() - timedelta(days = 1)).strftime('%Y-%m-%d')
day = '2011-05-01'

dataCompany = json.loads(r.get('dataCompany')) 

tempkWhFridge=0
tempkWhDryer=0
tempkWhMicrowave=0
tempkWhDishwasher=0
tempkWhElectricoven=0
tempkWhHob=0
tempkWhHeater=0

typePrediction='FHMM'

if predictions[typePrediction].loc[day+' 00:00:00':day+' 23:59:00'].index.size > 0:
    if appliances['Refrigerator']==True:
        for predictionTempFridge in predictions[typePrediction].loc[day+' 00:00:00':day+' 23:59:00'].Fridge:
            tempkWhFridge += ((predictionTempFridge/1000)*0.01666666667)
    if appliances['Dryer']==True:
        for predictionTempDryer in predictions[typePrediction].loc[day+' 00:00:00':day+' 23:59:00']["Washer dryer"]:
            tempkWhDryer += ((predictionTempDryer/1000)*0.01666666667)
    if appliances['Microwave']==True:
        for predictionTempMicrowave in predictions[typePrediction].loc[day+' 00:00:00':day+' 23:59:00'].Microwave:
            tempkWhMicrowave += ((predictionTempMicrowave/1000)*0.01666666667)
    if appliances['Dishwasher']==True:
        for predictionTempDishwasher in predictions[typePrediction].loc[day+' 00:00:00':day+' 23:59:00']["Dish washer"]:
            tempkWhDishwasher += ((predictionTempDishwasher/1000)*0.01666666667)
    if appliances['Oven']==True:
        for predictionTempElectricoven in predictions[typePrediction].loc[day+' 00:00:00':day+' 23:59:00']["Electric oven"]:
            tempkWhElectricoven += ((predictionTempElectricoven/1000)*0.01666666667)
    if appliances['Electric Hob']==True:
        for predictionTempHob in predictions[typePrediction].loc[day+' 00:00:00':day+' 23:59:00']["Electric stove"]:
            tempkWhHob += ((predictionTempHob/1000)*0.01666666667)
    if appliances['Electric heater']==True:
        for predictionTempHeater in predictions[typePrediction].loc[day+' 00:00:00':day+' 23:59:00']["Electric space heater"]:
            tempkWhHeater += ((predictionTempHeater/1000)*0.01666666667)

    data_setAppliances = []
    max_kWAll = tempkWhFridge+tempkWhDryer+tempkWhMicrowave+tempkWhDishwasher+tempkWhElectricoven+tempkWhHob+tempkWhHeater
    cost_kWh = float(dataCompany['pricekWh'])

    if appliances['Refrigerator']==True:
        data_setAppliances.append({"kWh":round(tempkWhFridge,3),"appliance":"Refrigerator","percent":math.ceil(((tempkWhFridge/max_kWAll)*100)),"cost":round((tempkWhFridge*cost_kWh),3)})
    if appliances['Dryer']==True:
        data_setAppliances.append({"kWh":round(tempkWhDryer,3),"appliance":"Dryer","percent":math.ceil(((tempkWhDryer/max_kWAll)*100)),"cost":round((tempkWhDryer*cost_kWh),3)})
    if appliances['Microwave']==True:
        data_setAppliances.append({"kWh":round(tempkWhMicrowave,3),"appliance":"Microwave","percent":math.ceil(((tempkWhMicrowave/max_kWAll)*100)),"cost":round((tempkWhMicrowave*cost_kWh),3)})
    if appliances['Dishwasher']==True:
        data_setAppliances.append({"kWh":round(tempkWhDishwasher,3),"appliance":"Dishwasher","percent":math.ceil(((tempkWhDishwasher/max_kWAll)*100)),"cost":round((tempkWhDishwasher*cost_kWh),3)})
    if appliances['Oven']==True:
        data_setAppliances.append({"kWh":round(tempkWhElectricoven,3),"appliance":"Oven","percent":math.ceil(((tempkWhElectricoven/max_kWAll)*100)),"cost":round((tempkWhElectricoven*cost_kWh),3)})
    if appliances['Electric Hob']==True:
        data_setAppliances.append({"kWh":round(tempkWhHob,3),"appliance":"Electric Hob","percent":math.ceil(((tempkWhHob/max_kWAll)*100)),"cost":round((tempkWhHob*cost_kWh),3)})
    if appliances['Electric heater']==True:
        data_setAppliances.append({"kWh":round(tempkWhHeater,3),"appliance":"Electric heater","percent":math.ceil(((tempkWhHeater/max_kWAll)*100)),"cost":round((tempkWhHeater*cost_kWh),3)})

    json_dumpAppliances = json.dumps(data_setAppliances)
    r.set('consumptionAppliances',json_dumpAppliances) 


In [ ]:
#print(predictions[typePrediction].loc[day+' 00:00:00':day+' 23:59:00'])

resultPredictionsAppliances = predictions[typePrediction].loc[day+' 00:00:00':day+' 23:59:00'].to_json(orient="index")
parsed = json.loads(resultPredictionsAppliances)
jsonOBJ_AppliancesToday = json.dumps(parsed, indent=1)
r.set('consumptionAppliancesToday',jsonOBJ_AppliancesToday)

#print(jsonOBJ_AppliancesToday)




In [ ]:
monthT = date.today().strftime('%m')
monthY = (date.today() - timedelta(days = 1)).strftime('%m')

kWhMonthRefrigerator=0
kWhMonthDryer=0
kWhMonthMicrowave=0
kWhMonthDishwasher=0
kWhMonthOven=0
kWhMonthElectricHob=0
kWhMonthElectricheater=0
kWhTotalMonth=0

if monthT==monthY:
    consumptionAppliancesMonth = json.loads(r.get('consumptionAppliancesMonth')) 
    
    for key in consumptionAppliancesMonth:
        if appliances['Refrigerator']==True and key['appliance']=='Refrigerator':
            key['kWh']+=round(tempkWhFridge,3)
            kWhTotalMonth+=key['kWh']
            kWhMonthRefrigerator=key['kWh']
            key['cost']+=round((tempkWhFridge*cost_kWh),3)
            
        if appliances['Dryer']==True and key['appliance']=='Dryer':
            key['kWh']+=round(tempkWhDryer,3)
            kWhTotalMonth+=key['kWh']
            kWhMonthDryer=key['kWh']
            key['cost']+=round((tempkWhDryer*cost_kWh),3)
            
        if appliances['Microwave']==True and key['appliance']=='Microwave':
            key['kWh']+=round(tempkWhMicrowave,3)
            kWhTotalMonth+=key['kWh']
            kWhMonthMicrowave=key['kWh']
            key['cost']+=round((tempkWhMicrowave*cost_kWh),3)
            
        if appliances['Dishwasher']==True and key['appliance']=='Dishwasher':
            key['kWh']+=round(tempkWhDishwasher,3)
            kWhTotalMonth+=key['kWh']
            kWhMonthDishwasher=key['kWh']
            key['cost']+=round((tempkWhDishwasher*cost_kWh),3)
            
        if appliances['Oven']==True and key['appliance']=='Oven':
            key['kWh']+=round(tempkWhElectricoven,3)
            kWhTotalMonth+=key['kWh']
            kWhMonthOven=key['kWh']
            key['cost']+=round((tempkWhElectricoven*cost_kWh),3)
        
        if appliances['Electric Hob']==True and key['appliance']=='Electric Hob':
            key['kWh']+=round(tempkWhHob,3)
            kWhTotalMonth+=key['kWh']
            kWhMonthElectricHob=key['kWh']
            key['cost']+=round((tempkWhHob*cost_kWh),3)
    
        if appliances['Electric heater']==True and key['appliance']=='Electric heater':
            key['kWh']+=round(tempkWhHeater,3)
            kWhTotalMonth+=key['kWh']
            kWhMonthElectricheater=key['kWh']
            key['cost']+=round((tempkWhHeater*cost_kWh),3)

    for key in consumptionAppliancesMonth:
        if appliances['Refrigerator']==True and key['appliance']=='Refrigerator':
            key['percent']=math.ceil(((kWhMonthRefrigerator/kWhTotalMonth)*100))
            
        if appliances['Dryer']==True and key['appliance']=='Dryer':
            key['percent']=math.ceil(((kWhMonthDryer/kWhTotalMonth)*100))
            
        if appliances['Microwave']==True and key['appliance']=='Microwave':
            key['percent']=math.ceil(((kWhMonthMicrowave/kWhTotalMonth)*100))
            
        if appliances['Dishwasher']==True and key['appliance']=='Dishwasher':
            key['percent']=math.ceil(((kWhMonthDishwasher/kWhTotalMonth)*100))
            
        if appliances['Oven']==True and key['appliance']=='Oven':
            key['percent']=math.ceil(((kWhMonthOven/kWhTotalMonth)*100))
        
        if appliances['Electric Hob']==True and key['appliance']=='Electric Hob':
            key['percent']=math.ceil(((kWhMonthElectricHob/kWhTotalMonth)*100))
    
        if appliances['Electric heater']==True and key['appliance']=='Electric heater':
            key['percent']=math.ceil(((kWhMonthElectricheater/kWhTotalMonth)*100))
            
    json_dumpAppliancesMonth2 = json.dumps(consumptionAppliancesMonth)
    r.set('consumptionAppliancesMonth',json_dumpAppliancesMonth2)
    
else:
    data_setAppliancesMonth = []
    if appliances['Refrigerator']==True:
        data_setAppliancesMonth.append({"kWh":round(tempkWhFridge,3),"appliance":"Refrigerator","percent":math.ceil(((tempkWhFridge/max_kWAll)*100)),"cost":round((tempkWhFridge*cost_kWh),3)})
    if appliances['Dryer']==True:
        data_setAppliancesMonth.append({"kWh":round(tempkWhDryer,3),"appliance":"Dryer","percent":math.ceil(((tempkWhDryer/max_kWAll)*100)),"cost":round((tempkWhDryer*cost_kWh),3)})
    if appliances['Microwave']==True:
        data_setAppliancesMonth.append({"kWh":round(tempkWhMicrowave,3),"appliance":"Microwave","percent":math.ceil(((tempkWhMicrowave/max_kWAll)*100)),"cost":round((tempkWhMicrowave*cost_kWh),3)})
    if appliances['Dishwasher']==True:
        data_setAppliancesMonth.append({"kWh":round(tempkWhDishwasher,3),"appliance":"Dishwasher","percent":math.ceil(((tempkWhDishwasher/max_kWAll)*100)),"cost":round((tempkWhDishwasher*cost_kWh),3)})
    if appliances['Oven']==True:
        data_setAppliancesMonth.append({"kWh":round(tempkWhElectricoven,3),"appliance":"Oven","percent":math.ceil(((tempkWhElectricoven/max_kWAll)*100)),"cost":round((tempkWhElectricoven*cost_kWh),3)})
    if appliances['Electric Hob']==True:
        data_setAppliancesMonth.append({"kWh":round(tempkWhHob,3),"appliance":"Electric Hob","percent":math.ceil(((tempkWhHob/max_kWAll)*100)),"cost":round((tempkWhHob*cost_kWh),3)})
    if appliances['Electric heater']==True:
        data_setAppliancesMonth.append({"kWh":round(tempkWhHeater,3),"appliance":"Electric heater","percent":math.ceil(((tempkWhHeater/max_kWAll)*100)),"cost":round((tempkWhHeater*cost_kWh),3)})

    json_dumpAppliancesMonth = json.dumps(data_setAppliancesMonth)
    r.set('consumptionAppliancesMonth',json_dumpAppliancesMonth) 
    


In [ ]:
yearT = date.today().strftime('%Y')
yearY = (date.today() - timedelta(days = 1)).strftime('%Y')

kWhYearRefrigerator=0
kWhYearDryer=0
kWhYearMicrowave=0
kWhYearDishwasher=0
kWhYearOven=0
kWhYearElectricHob=0
kWhYearElectricheater=0
kWhTotalYear=0

if yearT==yearY:
    consumptionAppliancesYear = json.loads(r.get('consumptionAppliancesYear')) 
    
    for key in consumptionAppliancesYear:
        if appliances['Refrigerator']==True and key['appliance']=='Refrigerator':
            key['kWh']+=round(tempkWhFridge,3)
            kWhTotalYear+=key['kWh']
            kWhYearRefrigerator=key['kWh']
            key['cost']+=round((tempkWhFridge*cost_kWh),3)
            
        if appliances['Dryer']==True and key['appliance']=='Dryer':
            key['kWh']+=round(tempkWhDryer,3)
            kWhTotalYear+=key['kWh']
            kWhYearDryer=key['kWh']
            key['cost']+=round((tempkWhDryer*cost_kWh),3)
            
        if appliances['Microwave']==True and key['appliance']=='Microwave':
            key['kWh']+=round(tempkWhMicrowave,3)
            kWhTotalYear+=key['kWh']
            kWhYearMicrowave=key['kWh']
            key['cost']+=round((tempkWhMicrowave*cost_kWh),3)
            
        if appliances['Dishwasher']==True and key['appliance']=='Dishwasher':
            key['kWh']+=round(tempkWhDishwasher,3)
            kWhTotalYear+=key['kWh']
            kWhYearDishwasher=key['kWh']
            key['cost']+=round((tempkWhDishwasher*cost_kWh),3)
            
        if appliances['Oven']==True and key['appliance']=='Oven':
            key['kWh']+=round(tempkWhElectricoven,3)
            kWhTotalYear+=key['kWh']
            kWhYearOven=key['kWh']
            key['cost']+=round((tempkWhElectricoven*cost_kWh),3)
        
        if appliances['Electric Hob']==True and key['appliance']=='Electric Hob':
            key['kWh']+=round(tempkWhHob,3)
            kWhTotalYear+=key['kWh']
            kWhYearElectricHob=key['kWh']
            key['cost']+=round((tempkWhHob*cost_kWh),3)
    
        if appliances['Electric heater']==True and key['appliance']=='Electric heater':
            key['kWh']+=round(tempkWhHeater,3)
            kWhTotalYear+=key['kWh']
            kWhYearElectricheater=key['kWh']
            key['cost']+=round((tempkWhHeater*cost_kWh),3)

    for key in consumptionAppliancesYear:
        if appliances['Refrigerator']==True and key['appliance']=='Refrigerator':
            key['percent']=math.ceil(((kWhYearRefrigerator/kWhTotalYear)*100))
            
        if appliances['Dryer']==True and key['appliance']=='Dryer':
            key['percent']=math.ceil(((kWhYearDryer/kWhTotalYear)*100))
            
        if appliances['Microwave']==True and key['appliance']=='Microwave':
            key['percent']=math.ceil(((kWhYearMicrowave/kWhTotalYear)*100))
            
        if appliances['Dishwasher']==True and key['appliance']=='Dishwasher':
            key['percent']=math.ceil(((kWhYearDishwasher/kWhTotalYear)*100))
            
        if appliances['Oven']==True and key['appliance']=='Oven':
            key['percent']=math.ceil(((kWhYearOven/kWhTotalYear)*100))
        
        if appliances['Electric Hob']==True and key['appliance']=='Electric Hob':
            key['percent']=math.ceil(((kWhYearElectricHob/kWhTotalYear)*100))
    
        if appliances['Electric heater']==True and key['appliance']=='Electric heater':
            key['percent']=math.ceil(((kWhYearElectricheater/kWhTotalYear)*100))
            
    json_dumpAppliancesYear2 = json.dumps(consumptionAppliancesYear)
    r.set('consumptionAppliancesYear',json_dumpAppliancesYear2)
    
else:
    data_setAppliancesYear = []
    if appliances['Refrigerator']==True:
        data_setAppliancesYear.append({"kWh":round(tempkWhFridge,3),"appliance":"Refrigerator","percent":math.ceil(((tempkWhFridge/max_kWAll)*100)),"cost":round((tempkWhFridge*cost_kWh),3)})
    if appliances['Dryer']==True:
        data_setAppliancesYear.append({"kWh":round(tempkWhDryer,3),"appliance":"Dryer","percent":math.ceil(((tempkWhDryer/max_kWAll)*100)),"cost":round((tempkWhDryer*cost_kWh),3)})
    if appliances['Microwave']==True:
        data_setAppliancesYear.append({"kWh":round(tempkWhMicrowave,3),"appliance":"Microwave","percent":math.ceil(((tempkWhMicrowave/max_kWAll)*100)),"cost":round((tempkWhMicrowave*cost_kWh),3)})
    if appliances['Dishwasher']==True:
        data_setAppliancesYear.append({"kWh":round(tempkWhDishwasher,3),"appliance":"Dishwasher","percent":math.ceil(((tempkWhDishwasher/max_kWAll)*100)),"cost":round((tempkWhDishwasher*cost_kWh),3)})
    if appliances['Oven']==True:
        data_setAppliancesYear.append({"kWh":round(tempkWhElectricoven,3),"appliance":"Oven","percent":math.ceil(((tempkWhElectricoven/max_kWAll)*100)),"cost":round((tempkWhElectricoven*cost_kWh),3)})
    if appliances['Electric Hob']==True:
        data_setAppliancesYear.append({"kWh":round(tempkWhHob,3),"appliance":"Electric Hob","percent":math.ceil(((tempkWhHob/max_kWAll)*100)),"cost":round((tempkWhHob*cost_kWh),3)})
    if appliances['Electric heater']==True:
        data_setAppliancesYear.append({"kWh":round(tempkWhHeater,3),"appliance":"Electric heater","percent":math.ceil(((tempkWhHeater/max_kWAll)*100)),"cost":round((tempkWhHeater*cost_kWh),3)})

    json_dumpAppliancesYear = json.dumps(data_setAppliancesYear)
    r.set('consumptionAppliancesYear',json_dumpAppliancesYear) 